In [12]:
import ee
import json
import pandas_gbq as gbq
from shapely.geometry import shape



In [13]:
def json_to_wkt(geometry):
    """
    Convert JSON geometry to WKT format.
    
    Args:
        json_geometry (dict): JSON geometry object.
    
    Returns:
        str: WKT representation of the geometry.
    """
    shapely_geometry = shape(json.loads(geometry))
    return shapely_geometry.wkt


In [3]:
service_account = 'austinbreunig@ee-obia-waterbodies.iam.gserviceaccount.com'
credentials = ee.ServiceAccountCredentials(service_account, 'ee-obia-waterbodies-33e289809d3b.json')
ee.Initialize(credentials)

In [4]:
sql = """
SELECT * FROM `ee-obia-waterbodies.ee_bq_test.ss_total_cat_depths`
"""

df = gbq.read_gbq(sql, project_id='ee-obia-waterbodies', dialect='standard')

Downloading: 100%|██████████|


In [5]:
df.head()

,geom,ID,RISK_VALUE,SURGE_RISK,FIPSSTCO,CAT1DEP,CAT2DEP,CAT3DEP,CAT4DEP,CAT5DEP,total_cat_
0,"{""type"":""Polygon"",""coordinates"":[[[-95.4399074...",17056189,4,Very High,48039,0,5,10,15,18,33
1,"{""type"":""Polygon"",""coordinates"":[[[-95.4374074...",17056827,4,Very High,48039,0,5,10,15,18,33
2,"{""type"":""Polygon"",""coordinates"":[[[-95.4374074...",17057446,4,Very High,48039,0,5,10,15,19,34
3,"{""type"":""Polygon"",""coordinates"":[[[-95.4407407...",17057591,4,Very High,48039,0,5,10,15,19,34
4,"{""type"":""Polygon"",""coordinates"":[[[-95.4396296...",17057779,4,Very High,48039,0,5,10,15,19,34


In [14]:
df['geom'] = df['geom'].apply(json_to_wkt)

In [15]:
import geemap

In [17]:
df.head()

,geom,ID,RISK_VALUE,SURGE_RISK,FIPSSTCO,CAT1DEP,CAT2DEP,CAT3DEP,CAT4DEP,CAT5DEP,total_cat_
0,POLYGON ((-95.43990740699996 28.96675925900007...,17056189,4,Very High,48039,0,5,10,15,18,33
1,POLYGON ((-95.43740740699997 28.96787037000007...,17056827,4,Very High,48039,0,5,10,15,18,33
2,POLYGON ((-95.43740740699997 28.96759259300006...,17057446,4,Very High,48039,0,5,10,15,19,34
3,POLYGON ((-95.44074074099996 28.96703703700006...,17057591,4,Very High,48039,0,5,10,15,19,34
4,POLYGON ((-95.43962962999994 28.96666666700002...,17057779,4,Very High,48039,0,5,10,15,19,34


In [19]:
df1 = df.iloc[0]['geom']
df1

'POLYGON ((-95.43990740699996 28.96675925900007, -95.44009259299997 28.96675925900007, -95.44009259299997 28.96666666700002, -95.44064814799998 28.96666666700002, -95.44064814799998 28.96657407400005, -95.43990740699996 28.96657407400005, -95.43962962999994 28.96657407400005, -95.43962962999994 28.96648148100007, -95.43962962999994 28.96638888900002, -95.43990740699996 28.96638888900002, -95.43990740699996 28.96648148100007, -95.44009259299997 28.96648148100007, -95.44009259299997 28.96638888900002, -95.44037036999998 28.96638888900002, -95.44037036999998 28.96629629600005, -95.43962962999994 28.96629629600005, -95.43935185199994 28.96629629600005, -95.43935185199994 28.96620370400006, -95.43944444399993 28.96620370400006, -95.43944444399993 28.96611111100003, -95.43981481499998 28.96611111100003, -95.43981481499998 28.96601851900004, -95.43935185199994 28.96601851900004, -95.43935185199994 28.96583333300003, -95.43962962999994 28.96583333300003, -95.43962962999994 28.96592592600006, -

In [20]:
aoi = ee.Geometry.Polygon(df1)

EEException: Invalid geometry.

In [16]:
Map = geemap.Map()
Map.centerObject(df, 12);
Map.setOptions(mapTypeId='HYBRID', styles={}, types=[])
Map.addLayer(df, {"color":"blue"});
Map

Exception: ee_object must be an instance of one of ee.Geometry, ee.FeatureCollection, ee.Image, or ee.ImageCollection.

In [ ]:
df.head()